In [ ]:
import pickle as pkl
import pandas as pd

In [ ]:
with open("./training_data_pandas.pkl", "br") as fh:
    data = pkl.load(fh)

In [ ]:
# TASK: some data exloration

# what kind object is "data"?
# what columns are there?
# what is the list of unique station names?

# rsds_stations is the in-situ observations in W/m**2
# integral_of_surface_downwelling_shortwave_flux_in_air_wrt_time_nora3 is the nora3 estimate
# and surface_solar_radiation_downwards_era5 the era5 one
# these are in Wh/m**2 (accumulated hourly values); create rsds_era5 and rsds_nora3 columns in data with the same normalization as rsds_stations

# plot rsds_stations for 1 station; what can you observe?
# what is the distribution of data in the rsds_stations column for the station? Is this a well balanced dataset? What could you do to make it "better balanced"?

In [ ]:
data

In [ ]:
data.columns

In [ ]:
data.name.unique()

In [ ]:
data["rsds_nora3"] = data["integral_of_surface_downwelling_shortwave_flux_in_air_wrt_time_nora3"] / 3600.0

In [ ]:
data["rsds_era5"] = data["surface_solar_radiation_downwards_era5"] / 3600.0

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# %matplotlib widget
%matplotlib tk

crrt_data = data[data["name"] == "OSLO - BLINDERN"]

plt.figure()
plt.plot(crrt_data["time_stations"], crrt_data["rsds_stations"])
plt.ylabel("W/m**2")
plt.show()

In [ ]:
# %matplotlib widget
%matplotlib tk

crrt_data = data[data["name"] == "TRONDHEIM - GLØSHAUGEN"]

plt.figure()
plt.plot(crrt_data["time_stations"], crrt_data["rsds_stations"])
plt.ylabel("W/m")

In [ ]:
%matplotlib tk

crrt_data = data[data["name"] == "TRONDHEIM - GLØSHAUGEN"]

plt.figure()
plt.hist(crrt_data["rsds_stations"], bins=300)
plt.show()

In [ ]:
##################################################################################################################

In [ ]:
# TASK: prepare the data

# Keep only the data where:
# “full_valid_flag” is True
# rsds_stations > 50.0
# rsds_nora3 > 50.0

# split between the training and validation datasets; use as a validation dataset the specific stations:
# "DOVRE-LANNEM",
# "TJØLLING",
# "GRAN",
# "RAKKESTAD",
# "ÅS",
# make *really* sure that the stations are well separated between the two, this is really important!

In [ ]:
import numpy as np

In [ ]:
valid_indexes = np.logical_and.reduce(
    (
        data["full_valid_flag"] == True,
        data["rsds_stations"] > 50.0,  # only use the cases when there is actually sun energy (not night)
        data["rsds_nora3"] > 50.0,  # only use the cases when there is actually sun energy (not night)
    )
)

In [ ]:
valid_data = data.loc[valid_indexes].reset_index()
print(f"size of valid_pd_all_data: {len(valid_data):,}")

In [ ]:
list_stations_for_validation = [
    "DOVRE-LANNEM",
    "TJØLLING",
    "GRAN",
    "RAKKESTAD",
    "ÅS",
]

In [ ]:
list_name_match_validation = [valid_data["name"] == st for st in list_stations_for_validation]
tuple_name_match_validation = tuple(list_name_match_validation)

# list our conditions to be a validation station
valid_indexes_validation = np.logical_or.reduce(
    tuple_name_match_validation
)

data_validation = valid_data.loc[valid_indexes_validation].reset_index().squeeze()
data_training = valid_data.loc[np.logical_not(valid_indexes_validation)].reset_index().squeeze()

In [ ]:
print(f"{len(data_validation) = }")
print(f"{len(data_training) = }")
print(f"{len(data_validation) + len(data_training) = }")

In [ ]:
data_validation.name.unique()

In [ ]:
data_training.name.unique()

In [ ]:
##################################################################################################################

In [ ]:
# TASK: investigate the general statistics of the full, training, and validation datasets, and the characteristics of the different data inputs

# print the MAE, RMSE, relative variance level, between the rsds_nora3, rsds_era5, SSI_value_cmsaf_sis
# and the rsds_stations

# you can use the following code as inspiration:

def print_stats(data_in, estimate_name):
    rsds = data_in["rsds_stations"].to_numpy()
    estimate = data_in[estimate_name].to_numpy()
    
    mae = np.mean(np.abs(rsds-estimate))
    rmse = np.sqrt(np.mean((rsds-estimate)**2))
    variance_ratio = np.std(estimate) / np.std(rsds)

    print(f"{estimate_name = }")
    print(f"{mae = }")
    print(f"{rmse = }")
    print(f"{variance_ratio = }")
    print("")

# do this for both the full dataset, and for the training and validation datasets individually

# plot and interpret the data in a Taylor plot
# you can use the example code as an inspiration:

import skill_metrics as sm
from skill_metrics.check_taylor_stats import check_taylor_stats
import time

def get_taylor_plot_stats(data_in, field_label_name, field_model_name):
    field_label_np = data_in.loc[:, field_label_name].to_numpy().astype(np.float32)
    field_model_np = data_in.loc[:, field_model_name].to_numpy().astype(np.float32)
    
    sdev_ref_label = np.std(field_label_np)
    field_label_np_normalized = field_label_np / sdev_ref_label
    field_model_np_normalized = field_model_np / sdev_ref_label

    sdev = np.std(field_model_np_normalized)
    crmsd = sm.centered_rms_dev(field_label_np_normalized, field_model_np_normalized)
    ccoef = np.corrcoef(field_label_np_normalized, field_model_np_normalized)[0,1]
    
    return (sdev, crmsd, ccoef)


plt.rc('font', size=17)

plt.figure(num=1, figsize=(16, 12))

first_Taylor_diagram = True

list_colors = ["r", "k", "y", "b", "g", "m", "c", "darkorange", "lime", "paleturquoise", "gold"]
dict_label = {"observations": "c"}

data_to_use = data_validation

init_sdev, init_crmsd, init_ccoef = get_taylor_plot_stats(data_to_use, "rsds_stations", "rsds_stations")

list_columns_model = [
    "rsds_nora3",
    "rsds_era5",
    "SSI_value_cmsaf_sis",
]

for crrt_color, crrt_model in zip(list_colors[0:len(list_columns_model)], list_columns_model):
    dict_label[crrt_model] = crrt_color
    
    crrt_list_sdev = [init_sdev]
    crrt_list_crmsd = [init_crmsd]
    crrt_list_ccoef = [init_ccoef]
    
    list_stations_to_use = data_to_use.name.unique()
    
    for crrt_station in list_stations_to_use:
        
        pd_crrt_station = data_to_use[data_to_use["name"] == crrt_station]

        crrt_sdev, crrt_crmsd, crrt_ccoef = get_taylor_plot_stats(pd_crrt_station, "rsds_stations", crrt_model)
        crrt_list_sdev.append(crrt_sdev)
        crrt_list_crmsd.append(crrt_crmsd)
        crrt_list_ccoef.append(crrt_ccoef)
        
    crrt_sdev = np.array(crrt_list_sdev)
    crrt_crmsd = np.array(crrt_list_crmsd)
    crrt_ccoef = np.array(crrt_list_ccoef)
    
    if first_Taylor_diagram:
        sm.taylor_diagram(crrt_sdev, crrt_crmsd, crrt_ccoef, markercolor=crrt_color, markersize=4)
    else:
        sm.taylor_diagram(crrt_sdev, crrt_crmsd, crrt_ccoef, overlay = 'on', markercolor=crrt_color, markerLabel=dict_label, markersize=5)
    
    first_Taylor_diagram = False
    
    pd_to_aggregate = pd.DataFrame()
    
    for crrt_station in list_stations_to_use:
        pd_crrt_station = data_to_use[data_to_use["name"] == crrt_station]
        if len(pd_crrt_station) < 20:
            continue
        pd_to_aggregate = pd.concat([pd_to_aggregate, pd_crrt_station], ignore_index=True)
    
    # inspired from https://github.com/PeterRochford/SkillMetrics/blob/master/skill_metrics/taylor_diagram.py to put in the correct polar coordinates
    crrt_sdev, crrt_crmsd, crrt_ccoef = get_taylor_plot_stats(pd_to_aggregate, "rsds_stations", crrt_model)
    crrt_sdev = np.array([crrt_sdev])
    crrt_crmsd = np.array([crrt_crmsd])
    crrt_ccoef = np.array([crrt_ccoef])
    check_taylor_stats(crrt_sdev, crrt_crmsd, crrt_ccoef, 0.01)
    rho, theta = crrt_sdev, np.arccos(crrt_ccoef)
    
    plt.scatter(rho * np.cos(theta), rho * np.sin(theta), s=[512], c=[crrt_color], marker="X", label=crrt_model)
    
plt.scatter([1.0], [0.0], s=[512], c=["c"], label="truth")
plt.legend(bbox_to_anchor=(1.25, 1.25), loc='upper right')
plt.tight_layout()